In [1]:
import pandas as pd, numpy as np
import itertools, requests, json

In [2]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')
lm=pd.read_excel('data/szekelyhon_locs_curated.xlsx')

In [3]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [4]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]
lm["uid"] = lm["title"].map(str) + lm["content"]

In [5]:
dc=dc.set_index('uid')[['relevant','severity','deaths','duplicate']]
dm=dm.set_index('uid')
lm=lm.set_index('uid')[['predicted_locs','actual_locs']]

In [6]:
dmi=dm.loc[dc.index.dropna()]

In [7]:
df=dmi.join(dc).drop_duplicates()

In [8]:
dl=df.join(lm)

In [9]:
len(dl)

29

Working df ready. Now infer location.

In [10]:
to_geo=[i.split(',') for i in dl['actual_locs'].unique() if str(i)!='nan']

In [11]:
to_geo=[i.strip() for i in list(itertools.chain.from_iterable(to_geo))]

Load HU names

In [12]:
coords=json.loads(open('geo/huncoords.json','r').read())
megyek=json.loads(open('geo/megyek.json','r').read())

In [13]:
data=[]
header=['index']
for i in dl.iteritems():
    header.append(i[0])
header.append('geo_loc')
header.append('latitude')
header.append('longitude')
header.append('id')
for i in dl.T.iteritems():
    dummy=[i[0]]
    for j in range(len(i[1])):
        if j==(len(i[1])-1):
            dummy.append(i[1][j])
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dtime=str(dummy[1])[:10]
            if dtime=='nan': dtime=u'N/A' 
            if str(i[1][j])!='nan':    
                for k in i[1][j].split(','):
                    tk=k.strip().replace('[','').replace(']','').replace("'",'')
                    dummy[-4]=tk
                    if tk in coords:
                        dummy[-3]=coords[tk][0]
                        dummy[-2]=coords[tk][1]
                    else:
                        dummy[-3]=''
                        dummy[-2]=''
                    #dummy[-1]=str(dummy[-3])+' ('+str(dummy[0]+1)+')\n'+dtime
                    dummy[-1]=str(dummy[-4])+', '+dtime
                    data.append(list(dummy))                    
            else:
                dummy[-1]=u'N/A' +', '+dtime
                data.append(list(dummy))
        else: 
            dummy.append(i[1][j])

In [14]:
df=pd.DataFrame(data)
df.columns=header
df

,index,Unnamed: 0,date,hudate,link,image,category,icategory,tags,title,...,relevant,severity,deaths,duplicate,predicted_locs,actual_locs,geo_loc,latitude,longitude,id
0,Medvét láttak Csíkszentkirályon – ez volt az é...,61,2021-02-05,2021. február 05.,https://szekelyhon.ro/aktualis/medvet-lattak-c...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,['Csíkszentkirály'],Medvét láttak Csíkszentkirályon – ez volt az é...,...,1,2,0,0,"['Csíkszentmihály', 'Csíkszentkirály']",Csíkszentkirály,Csíkszentkirály,46.31208,25.821061,"Csíkszentkirály, 61"
1,Miért jár a medve iskolába? – Tánczos Barna a ...,62,2021-02-05,2021. február 05.,https://szekelyhon.ro/aktualis/miert-jar-a-med...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Nézőpont', 'Székelyhon Tv']",Miért jár a medve iskolába? – Tánczos Barna a ...,...,1,0,0,0,"['Haraly', 'Nyomát']",NaN,,,,"N/A, 62"
2,"Medvebocsokat szedtek ki egy barlangból, vide...",63,2021-02-19,2021. február 19.,https://szekelyhon.ro/aktualis/medvebocsokat-s...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,['Gyergyószék'],"Medvebocsokat szedtek ki egy barlangból, vide...",...,1,2,0,0,NaN,NaN,,,,"N/A, 63"
3,Nem találták meg a medvebocsokatFolytatódik a ...,64,2021-02-22,2021. február 22.,https://szekelyhon.ro/aktualis/nem-talaltak-me...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Gyergyószék', 'medve']",Nem találták meg a medvebocsokat,...,1,0,0,0,NaN,NaN,,,,"N/A, 64"
4,Mennyi medve van hat év semmittevés után?Erre ...,65,2021-02-22,2021. február 22.,https://szekelyhon.ro/aktualis/mennyi-medve-va...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Háromszék', 'Környezetvédelem']",Mennyi medve van hat év semmittevés után?,...,1,0,0,0,Keresd,Sepsiszentgyörgy,Sepsiszentgyörgy,45.860937,25.78858,"Sepsiszentgyörgy, 65"
5,Elköltöztetik a Vásárhelyen sétáló medvétMaros...,66,2021-02-26,2021. február 26.,https://szekelyhon.ro/aktualis/elkoltoztetik-a...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Marosvásárhely', 'medve']",Elköltöztetik a Vásárhelyen sétáló medvét,...,1,0,0,0,Marosvásárhely,Marosvásárhely,Marosvásárhely,46.538586,24.551439,"Marosvásárhely, 66"
6,"Újra aktívak a medvék, de még békén hagyják a...",67,2021-03-04,2021. március 04.,https://szekelyhon.ro/aktualis/ujra-aktivak-a-...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['Hargita megye', 'medve']","Újra aktívak a medvék, de még békén hagyják a...",...,1,0,0,0,NaN,NaN,,,,"N/A, 67"
7,Gyergyószentmiklósi bevásárlóközpontnál járt e...,68,2021-03-09,2021. március 09.,https://szekelyhon.ro/aktualis/gyergyoszentmik...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Gyergyószentmiklós', 'medve']",Gyergyószentmiklósi bevásárlóközpontnál járt e...,...,1,2,0,0,"['Kerelő', 'Gyergyószentmiklós', 'Keresd']",Gyergyószentmiklós,Gyergyószentmiklós,46.721211,25.585528,"Gyergyószentmiklós, 68"
8,Ismét medvét láttak GyergyószentmiklósonMedve ...,69,2021-03-09,2021. március 09.,https://szekelyhon.ro/aktualis/ismet-medvet-la...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,['Gyergyószentmiklós'],Ismét medvét láttak Gyergyószentmiklóson,...,1,2,0,0,Gyergyószentmiklós,Gyergyószentmiklós,Gyergyószentmiklós,46.721211,25.585528,"Gyergyószentmiklós, 69"
9,Nem tágít Gyergyószentmiklósról a medveHáromsz...,70,2021-03-10,2021. március 10.,https://szekelyhon.ro/aktualis/nem-tagit-gyerg...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Gyergyószentmiklós', 'medve']",Nem tágít Gyergyószentmiklósról a medve,...,1,2,0,0,Gyergyószentmiklós,Gyergyószentmiklós,Gyergyószentmiklós,46.721211,25.585528,"Gyergyószentmiklós, 70"


Save data for loading in viz app

In [15]:
df['checked']=''
df['exact_loc']=0
df['source']='Székelyhon'

In [16]:
df=df[['checked',
    'date',
    'link',
    'image',
    'source',
    'title',
    'content',
    'relevant',
    'severity',
    'deaths',
    'duplicate',
    'geo_loc',
    'exact_loc',
    'latitude',
    'longitude']].set_index('checked')

In [17]:
#PowerBI exploration data
df.to_excel('data/szekelyhon_data.xlsx')